In [1]:
# imports 
from stable_baselines3 import DQN, PPO
import gymnasium as gym
import gym_gridworlds
import matplotlib.pyplot as plt
import pandas as pd
from stable_baselines3.common.evaluation import evaluate_policy
import os

In [2]:
# make environment
env = gym.make("Gym-Gridworlds/Empty-RandomStart-3x3-v0", no_stay=True, distance_reward=True)
# logging wrapper
os.makedirs("logs/", exist_ok=True)
os.makedirs("trained_models", exist_ok=True)

In [3]:
model = PPO(
    "MlpPolicy",
    env,
    # batch_size= , # minibatch size
    learning_rate=1e-4,
    # n_epochs = ,# 🤷‍♀️
    # gamma = , # discount factor
    # ent_coef= , # entropy coefficient for loss calculation
    # vf_coef= , # value function for loss coefficient
    # target_kl = ,# Limit the KL divergence between updates
    stats_window_size = 100
)

In [4]:
# train model, progress bar w tdqm
print("--- Starting Training ---")
model.learn(total_timesteps=25000, progress_bar=True)
print( "--- Training Finished ---")

/Users/alisonandrade/Desktop/gym_gridworlds/.venv/lib/python3.14/site-packages/rich/live.py:256: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

--- Starting Training ---


--- Training Finished ---


In [5]:
model.save("trained_models/3x3")

In [6]:
eval_env = gym.make("Gym-Gridworlds/Empty-RandomStart-3x3-v0", no_stay=True, distance_reward=True)
trained_model = PPO.load("trained_models/3x3")
mean_reward, std_reward = evaluate_policy(trained_model, eval_env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")


Mean reward: 0.80 +/- 0.20


/Users/alisonandrade/Desktop/gym_gridworlds/.venv/lib/python3.14/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
